In [32]:
import pandas as pd
import numpy as np
import spacy
from spacy_sentiws import spaCySentiWS
import re
import swifter
import time
import statistics

In [6]:
df = pd.read_csv('../../data/p_comment.csv')
df.head()

,ID_Posting,ID_Posting_Parent,ID_GodotObject,ID_CommunityIdentity,id_posting_status,Headline,Comment,createdAt,VoteNegative,VotePositive,merged_text,p_merged_text
0,1049746955,1049746300,2000114365338,581535,1.0,NaN,"Nö, noch um einiges schlimmer.",2020-02-10 13:39:17.460,False,True,"Nö, noch um einiges schlimmer. |",nö schlimm
1,1051784424,1051784054,2000116189812,632068,1.0,NaN,"Wieder einer, der nicht kapiert, dass Arbeit k...",2020-03-26 12:08:22.187,False,True,"Wieder einer, der nicht kapiert, dass Arbeit k...",kapieren Arbeit Gängelei notwendig Versorgung ...
2,1051784801,-1,2000116189812,600252,1.0,ich fasse es mit den worten von prof. falco zu...,nie mehr schule!,2020-03-26 12:13:29.653,True,False,nie mehr schule! | ich fasse es mit den worten...,nie mehr Schule fassen warten pro zusammen
3,1051785187,-1,2000116189812,701802,1.0,NaN,Das wird Monate dauern ihr Traumtänzer... Das ...,2020-03-26 12:19:13.047,True,False,Das wird Monate dauern ihr Traumtänzer... Das ...,Monat dauern Traumtänzer stehen einfach dafür
4,1051785575,1051784986,2000116189812,702957,1.0,Dann sind wir gefordert,und müssen in Massen auf dieStrassen.,2020-03-26 12:25:26.320,False,True,und müssen in Massen auf dieStrassen. | Dann s...,müssen Masse Diestrasse fordern


In [7]:
# exclude the found phrases _ and only consider single words
text = 'Monat dauern Traumtänzer stehen_einfach dafür'.replace('_', ' ');
nlp = spacy.load('de_core_news_md')
nlp.add_pipe('sentiws',config={'sentiws_path': '../../data/sentiws/'})
doc = nlp(text)
for token in doc:
    print('{}, {}, {}'.format(token.text, token._.sentiws, token.pos_))

Monat, None, NOUN
dauern, None, VERB
Traumtänzer, None, NOUN
stehen, None, VERB
einfach, None, ADV
dafür, None, ADV


In [8]:
def get_overall_sentiment(text):
    text = text.replace('_',' ')
    doc = nlp(text)
    sentiment_score = 0
    for token in doc:
        #print('{}, {}, {}'.format(token.text, token._.sentiws, token.pos_))
        sentiment_score += token._.sentiws or 0
    
    return sentiment_score / len(doc)

start = time.time()
df['sentiment'] = df['p_merged_text'].swifter.apply(lambda x: get_overall_sentiment(str(x)))
end = time.time()

print('processing time: ', end-start)

df.head()

Pandas Apply:   0%|          | 0/6485 [00:00<?, ?it/s]

processing time:  93.0818235874176


,ID_Posting,ID_Posting_Parent,ID_GodotObject,ID_CommunityIdentity,id_posting_status,Headline,Comment,createdAt,VoteNegative,VotePositive,merged_text,p_merged_text,sentiment
0,1049746955,1049746300,2000114365338,581535,1.0,NaN,"Nö, noch um einiges schlimmer.",2020-02-10 13:39:17.460,False,True,"Nö, noch um einiges schlimmer. |",nö schlimm,0.000000
1,1051784424,1051784054,2000116189812,632068,1.0,NaN,"Wieder einer, der nicht kapiert, dass Arbeit k...",2020-03-26 12:08:22.187,False,True,"Wieder einer, der nicht kapiert, dass Arbeit k...",kapieren Arbeit Gängelei notwendig Versorgung ...,0.000667
2,1051784801,-1,2000116189812,600252,1.0,ich fasse es mit den worten von prof. falco zu...,nie mehr schule!,2020-03-26 12:13:29.653,True,False,nie mehr schule! | ich fasse es mit den worten...,nie mehr Schule fassen warten pro zusammen,0.000000
3,1051785187,-1,2000116189812,701802,1.0,NaN,Das wird Monate dauern ihr Traumtänzer... Das ...,2020-03-26 12:19:13.047,True,False,Das wird Monate dauern ihr Traumtänzer... Das ...,Monat dauern Traumtänzer stehen einfach dafür,0.000000
4,1051785575,1051784986,2000116189812,702957,1.0,Dann sind wir gefordert,und müssen in Massen auf dieStrassen.,2020-03-26 12:25:26.320,False,True,und müssen in Massen auf dieStrassen. | Dann s...,müssen Masse Diestrasse fordern,0.000000


In [28]:
#weight the sentiment score based on the votes
# less negative than possitive comments
df1 = df.copy()
wh=2
wl=1
#df1.to_csv('../../data/feature/sentiws_comment.csv', encoding='utf-8', index=False)
df1['weight'] = np.where((df1['sentiment'] > 0) & (df1['VotePositive']), wh,
                        np.where((df1['sentiment'] < 0) & (df1['VotePositive']), wh, wl))
df1.head()

,ID_Posting,ID_Posting_Parent,ID_GodotObject,ID_CommunityIdentity,id_posting_status,Headline,Comment,createdAt,VoteNegative,VotePositive,merged_text,p_merged_text,sentiment,weight
0,1049746955,1049746300,2000114365338,581535,1.0,NaN,"Nö, noch um einiges schlimmer.",2020-02-10 13:39:17.460,False,True,"Nö, noch um einiges schlimmer. |",nö schlimm,0.000000,1
1,1051784424,1051784054,2000116189812,632068,1.0,NaN,"Wieder einer, der nicht kapiert, dass Arbeit k...",2020-03-26 12:08:22.187,False,True,"Wieder einer, der nicht kapiert, dass Arbeit k...",kapieren Arbeit Gängelei notwendig Versorgung ...,0.000667,2
2,1051784801,-1,2000116189812,600252,1.0,ich fasse es mit den worten von prof. falco zu...,nie mehr schule!,2020-03-26 12:13:29.653,True,False,nie mehr schule! | ich fasse es mit den worten...,nie mehr Schule fassen warten pro zusammen,0.000000,1
3,1051785187,-1,2000116189812,701802,1.0,NaN,Das wird Monate dauern ihr Traumtänzer... Das ...,2020-03-26 12:19:13.047,True,False,Das wird Monate dauern ihr Traumtänzer... Das ...,Monat dauern Traumtänzer stehen einfach dafür,0.000000,1
4,1051785575,1051784986,2000116189812,702957,1.0,Dann sind wir gefordert,und müssen in Massen auf dieStrassen.,2020-03-26 12:25:26.320,False,True,und müssen in Massen auf dieStrassen. | Dann s...,müssen Masse Diestrasse fordern,0.000000,1


In [54]:
## calculate weighterd median of the discussion sentiment
df2 = df1[['ID_Posting', 'ID_Posting_Parent','ID_GodotObject', 'createdAt', 'sentiment', 'weight']]
df2['sentiment'] = df2['sentiment'] * df2['weight']
df2.drop('weight', inplace=True, axis=1)

# method for calculating median of subconversation
def conversation_sentiment_scores(posting_id, sents, replies_df):
    # find all direct replies for the posting
    #print(posting_id)
    replies = replies_df[replies_df["ID_Posting_Parent"] == posting_id]
    #print(len(replies))
    if replies.empty:
        # Return the current depth if there are no replies
        return sents
    else:
        #sents.extend(replies['sentiment'])
        for row in replies.itertuples():
            sub = conversation_sentiment_scores(row.ID_Posting, [row.sentiment], replies_df)
            sents += sub
            
        return sents

    
    
# First, we need to filter out only the root comments
root_comments = df2[df2['ID_Posting_Parent'] == -1]

# Next, we need to filter out only the replies
replies = df2[df2['ID_Posting_Parent'] != -1]

#conversation_sentiment_scores(1052015173, [3], replies)
root_comments['overal_sentiment'] = root_comments.swifter.apply(lambda x: statistics.median(
    conversation_sentiment_scores(x.ID_Posting, [x.sentiment], replies)), axis=1)

root_comments.reset_index(drop=True, inplace=True)
print(root_comments.shape)
root_comments.head()

C:\Users\e11925939\AppData\Local\Temp\8\ipykernel_41756\32668361.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['sentiment'] = df2['sentiment'] * df2['weight']
C:\Users\e11925939\AppData\Local\Temp\8\ipykernel_41756\32668361.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.drop('weight', inplace=True, axis=1)


Pandas Apply:   0%|          | 0/2110 [00:00<?, ?it/s]

(2110, 6)


C:\Users\e11925939\AppData\Local\Temp\8\ipykernel_41756\32668361.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  root_comments['overal_sentiment'] = root_comments.swifter.apply(lambda x: statistics.median(


,ID_Posting,ID_Posting_Parent,ID_GodotObject,createdAt,sentiment,overal_sentiment
0,1051784801,-1,2000116189812,2020-03-26 12:13:29.653,0.000000,0.000000
1,1051785187,-1,2000116189812,2020-03-26 12:19:13.047,0.000000,0.000000
2,1051787674,-1,2000116189812,2020-03-26 12:54:17.667,-0.006926,-0.006926
3,1051793823,-1,2000116189812,2020-03-26 14:27:23.917,0.000000,0.000000
4,1051795844,-1,2000116189812,2020-03-26 14:57:20.867,0.000471,0.000471


In [53]:
root_comments.to_csv('../../data/feature/sentiws_comment.csv', encoding='utf-8', index=False)